# Simulate wrapped vortex shifts

In the configfile, you need:

```grey_pupils = True```  
```corona_type = 'wrapped_vortex'```

In [ ]:
import os
from astropy.io import fits
import matplotlib as mpl
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np

from Asterix import Asterix_root
from Asterix.main_THD import THD2
from Asterix.utils import create_experiment_dir, get_data_dir, read_parameter_file

In [ ]:
# Some setup for pretty plotting
mpl.rc('image', origin='lower',   # Put the origin in the lower left corner.
       interpolation=None)        # Do not interpolate between pixels in the display.

In [ ]:
data_dir = get_data_dir()
your_directory = Asterix_root
your_parameter_file_name = 'Example_param_file.ini'
parameter_file_path = os.path.join(your_directory, your_parameter_file_name)

In [ ]:
# Individual readings needed from parameter file
config = read_parameter_file(parameter_file_path)
simu_config = config["SIMUconfig"]
model_local_dir = os.path.join(data_dir, 'Model_local')

### Create THD2 simulator instance

Do not forget to set the grey pupil parameter in parameterfile to True.

In [ ]:
thd2 = THD2(parameter_file_path)

### Add aberrations to model

In [ ]:
# Phase upstream of the coronagraph (entrance pup)
phase_abb_up = thd2.generate_phase_aberr(simu_config, up_or_down='up', Model_local_dir=model_local_dir)

# Phase downstream of the coronagraph (Lyot stop)
phase_abb_do = thd2.generate_phase_aberr(simu_config, up_or_down='do', Model_local_dir=model_local_dir)

# Amplitude upstream of the coronagraph (entrance pup)
ampl_abb_up = thd2.generate_ampl_aberr(simu_config, Model_local_dir=model_local_dir)

### Create the wavefronts including the phase and amplitude aberrations
# WF in the testbed entrance pupil
input_wavefront = thd2.EF_from_phase_and_ampl(phase_abb=phase_abb_up, ampl_abb=ampl_abb_up)

# WF in the testbed Lyot stop
wavefront_in_LS = thd2.EF_from_phase_and_ampl(phase_abb=phase_abb_do)

### Direct PSF for normalization

In [ ]:
direct = thd2.todetector_intensity(in_contrast=True, entrance_EF=input_wavefront, EF_aberrations_introduced_in_LS=wavefront_in_LS,
                                   noFPM=True)
norm = direct.max()
print(norm)

plt.figure(figsize=(6, 6))
plt.imshow(direct/norm, cmap='inferno', norm=LogNorm(), origin='lower')
plt.colorbar()

### Coro PSF with flat wavefronts

In [ ]:
psf_coro = thd2.todetector_intensity(in_contrast=True, entrance_EF=input_wavefront,
                                     EF_aberrations_introduced_in_LS=wavefront_in_LS)

plt.figure(figsize=(6, 6))
plt.imshow(psf_coro/norm, cmap='inferno', norm=LogNorm(), origin='lower')
plt.colorbar()

### Coro PSF with applied DH

These are DM voltages from a WFS&C run in which the wrapped vortex FPM was perfectly centered. You can play with FPM shifts by restarting the kernel rerunning the entire notebook after adapting the Asterix scripts and configfile.

In [ ]:
data_out = '20221109_WV_shifts_phase'

In [ ]:
# With aberrations like above
experiment_dir = os.path.join(data_dir, 'Results', '20221108_14-09-40_My_fourth_experiment')
dh_iteration = 8

# Without aberrations
# experiment_dir = os.path.join(data_dir, 'Results', '20221108_17-00-47_My_fourth_experiment')
# dh_iteration = 11

In [ ]:
# Read DM commands to apply
dm1_voltages = fits.getdata(os.path.join(experiment_dir, 'DM1_voltages.fits'))[dh_iteration - 1]
dm3_voltages = fits.getdata(os.path.join(experiment_dir, 'DM3_voltages.fits'))[dh_iteration - 1]

# Concatenate into single array
dm_voltages = np.concatenate((dm1_voltages, dm3_voltages))

print(f'Numer of actuators in testbed: {thd2.number_act}')
print(f'Number of actuators read in: {dm_voltages.shape[0]}')

In [ ]:
# Calculate DH image
coro_dh = thd2.todetector_intensity(in_contrast=True, entrance_EF=input_wavefront,
                                    EF_aberrations_introduced_in_LS=wavefront_in_LS,
                                    voltage_vector=dm_voltages)

plt.figure(figsize=(6, 6))
plt.imshow(coro_dh/norm, cmap='inferno', norm=LogNorm(vmin=1e-8, vmax=1e-6), origin='lower')
plt.colorbar()

fname = '0x_02y'
plt.savefig(os.path.join(data_dir, 'Results', data_out, f'{fname}.pdf'))
fits.writeto(os.path.join(data_dir, 'Results', data_out, f'{fname}.fits'), coro_dh/norm, overwrite=True)

## Plotting

### Read and plot purely shifted images on a grid

In [ ]:
# imgs = []
# for i in range(4):
#     for j in range(4):
#         fname = f'{i}x_{j}y'
#         print(fname)
#         im = fits.getdata(os.path.join(data_dir, 'Results', data_out, 'shifts', 'fits', f'{fname}.fits'))
#         imgs.append(im)

In [ ]:
# plt.figure(figsize=(10, 10))
# for i in range(4):
#     for j in range(4):
#         plt.subplot(4, 4, 4*i + j + 1)
#         fname = f'{i}x_{j}y'
#         plt.imshow(imgs[4*i+j], cmap='inferno', norm=LogNorm(vmin=1e-8, vmax=1e-6), origin='lower')
#         plt.text(150, 350, fname, color='k', fontweight='bold', size=15)
#         plt.axis('off')
# plt.tight_layout()
# #plt.show()
# #plt.savefig(os.path.join(data_dir, 'Results', data_out, 'shifts', f'full_grid.pdf'))

### Read and plot purely inclined images on a grid

In [ ]:
# vals = [10, 13, 15, 16, 17, 18, 20, 22]
# imgs = []
# for deg in vals:
#     fname = f'incl_x_{deg}deg'
#     print(fname)
#     im = fits.getdata(os.path.join(data_dir, 'Results', data_out, 'inclinations',
#                                    'fits', f'{fname}.fits'))
#     imgs.append(im)

In [ ]:
# plt.figure(figsize=(18, 9))
# for i, deg in enumerate(vals):
#     plt.subplot(2, 4, i+1)
#     fname = f'{deg}deg'
#     plt.imshow(imgs[i], cmap='inferno', norm=LogNorm(vmin=1e-8, vmax=1e-6), origin='lower')
#     plt.text(150, 350, fname, color='k', fontweight='bold', size=25)
#     plt.axis('off')
# plt.tight_layout()
# #plt.show()
# #plt.savefig(os.path.join(data_dir, 'Results', data_out, 'inclinations', f'full_grid.pdf'))